In [1]:
import torch
import torchvision
import torchaudio 
import os
import torch.nn as nn
from os.path import isfile, join, isdir, exists
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [2]:
'''spec_dir = 'mel'
out_dir = 'mid'
years = ['2006']
ims = []
outs = []

for dirs in years:
    for f in os.listdir(join(spec_dir, dirs)):
        ims.append(join(join(spec_dir, dirs), f))
        onm = join(out_dir, dirs)
        outs.append(join(onm, f))
        break
    break

print(ims)
print(outs)
inn = torch.load(ims[0])
outt = torch.load(outs[0])

print(inn.shape, outt.shape)

print(type(inn), type(outt))'''


"spec_dir = 'mel'\nout_dir = 'mid'\nyears = ['2006']\nims = []\nouts = []\n\nfor dirs in years:\n    for f in os.listdir(join(spec_dir, dirs)):\n        ims.append(join(join(spec_dir, dirs), f))\n        onm = join(out_dir, dirs)\n        outs.append(join(onm, f))\n        break\n    break\n\nprint(ims)\nprint(outs)\ninn = torch.load(ims[0])\noutt = torch.load(outs[0])\n\nprint(inn.shape, outt.shape)\n\nprint(type(inn), type(outt))"

In [3]:
class MusicDataset(Dataset):

    def __init__(self, spec_dir, out_dir, years):
        self.spec_dir = spec_dir
        self.out_dir = out_dir
        self.ims = []
        self.labels = []

        for dirs in years:
            for f in os.listdir(join(spec_dir, dirs)):
                if(isfile(join(join(out_dir, dirs), f))):
                    self.ims.append(join(join(spec_dir, dirs), f))
                    onm = join(out_dir, dirs)
                    self.labels.append(join(onm, f))

    def __len__(self):
        return len(self.ims)

    def __getitem__(self, idx):
        return torch.load(self.ims[idx]), torch.load(self.labels[idx])

bs = 32
training_data = MusicDataset('mel/', 'mid/', ['2004', '2006', '2008', '2009', '2011', '2013', '2014'])
test_data = MusicDataset('mel/', 'mid/', ['2015', '2017', '2018'])
train_dataloader = DataLoader(training_data, batch_size=bs, shuffle=True, )
test_dataloader = DataLoader(test_data, batch_size=bs, shuffle=False)

In [4]:
# 229 x 1001 input
# 88 x 7681 output

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.ds1 = dsResConvBlock(1, 16)
        self.ds2 = dsResConvBlock(16, 64)
        self.cnn1 = nn.Conv2d(64, 64, 3, 1, padding='same')
        self.cnn2 = nn.Conv2d(64, 64, 3, 1, padding='same')
        self.us1 = usResConvBlock(64, 16)
        self.us2 = usResConvBlock(32, 4, out_size = (88, 7680+5))
        self.ReLU = nn.ReLU()

        self.linear = nn.Linear(4*6*88, 88)

    def forward(self, x):
        y = self.ds1(x)
        res1 = y
        y = self.ds2(y)
        res2 = y
        y = self.cnn1(y)
        y = self.cnn2(y)
        y = self.ReLU(y + res2)
        y = self.us1(y)
        # print(y.shape, res1.shape)
        y = torch.cat((res1, y), axis=1)
        y = self.us2(y)
            
        output = torch.zeros((y.shape[0], 88, 7680)).to('cuda')
        # print(y.shape)
        for i in range(7680):
            # print(y[:, :, :, i:i+6].shape)
            tp = y[:, :, :, i:i+6].reshape(-1, 4*6*88)
            # print(tp.shape)
            output[:, :, i] = nn.Sigmoid()(self.linear(tp))
        
        return output


class dsResConvBlock(nn.Module):
    def __init__(self, num_in, num_out):
        super(dsResConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_in, num_out, (3, 3), (1,1), padding='same')
        self.conv2 = nn.Conv2d(num_out, num_out, (3, 3), (1,1), padding='same')
        self.bn = nn.BatchNorm2d(num_out, momentum=0.9, eps=1e-5)
        self.mp = nn.MaxPool2d((2, 2), stride=(2, 2))
        self.ReLU = nn.ReLU()

    def forward(self, x):
        residual = self.ReLU(self.conv1(x))
        y = (self.conv2(residual))
        y = self.bn(y)
        y = self.ReLU(y + residual)
        y = self.mp(y)
        return y

class usResConvBlock(nn.Module):
    def __init__(self, num_in, num_out, out_size = (1,1)):
        super(usResConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_in, num_out, (3, 3), (1,1), padding='same')
        self.conv2 = nn.Conv2d(num_out, num_out, (3, 3), (1,1), padding='same')
        self.bn = nn.BatchNorm2d(num_out, momentum=0.9, eps=1e-5)
        self.ReLU = nn.ReLU()
        self.out_size = out_size

    def forward(self, x):
        # print("forward 1", x.shape)
        residual = self.ReLU(self.conv1(x))
        y = (self.conv2(residual))
        y = self.ReLU(y + residual)
        # print("forward 2", y.shape)
        if(self.out_size == (1,1)):
            # print("s\n")
            y = nn.Upsample(scale_factor = 2, mode='bilinear')(y)
        else:
            # print("a\n", self.out_size)
            y = nn.Upsample(size = self.out_size, mode='bilinear')(y)
        # print("foward 3")


        return y

def loss_func(x, y):
    return torch.mean(torch.abs(x-y))

In [5]:
print(len(train_dataloader))

1574


In [6]:
model = Model()
model = model.to('cuda')
lr = 3e-4
optim = torch.optim.Adam(lr = lr, params = model.parameters())
save_epochs = 1
val_print_ep = 5
n_epochs = 100
losses = []
for i in range(n_epochs):
    print("starting epoch", i)
    total_loss = 0
    batch = 0
    intlosses = []
    for X, y in train_dataloader:
        print("batch progress", batch, len(train_dataloader))
        batch += 1
        if(batch % 10 == 0):
            torch.save({
            'epoch': i,
            'batch': batch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': intlosses,
            }, 'intermediate_save.pt')
        
        X = torch.unsqueeze(X, axis = 1)
        X = X.to('cuda')
        # print(X.shape)
        y = y.to('cuda')
        optim.zero_grad()
        pred = model(X)
        loss = loss_func(y, pred)
        print(loss.item())
        intlosses.append(loss.item())
        print(intlosses)
        total_loss += loss.item()
        loss.backward()
        optim.step()
    loss.append(total_loss)
    print(total_loss)
    if (i % save_epochs == 0):
        torch.save({
            'epoch': i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': losses,
            }, 'saved_data.pt')

    if (i % val_print_ep == 0):
        with torch.no_grad():
            total_loss = 0
            for X, y in test_dataloader:
                X = torch.unsqueeze(X, axis = 1)
                X = X.to('cuda')
                y = y.to('cuda')
                pred = model(X)
                loss = loss_func(y, pred)
                total_loss += loss
        print("Test error after epoch " + str(i) + ": " + str(total_loss/len(test_dataloader)))

starting epoch 0
batch progress 0 1574
0.5133620500564575
[0.5133620500564575]
batch progress 1 1574
0.39571985602378845
[0.5133620500564575, 0.39571985602378845]
batch progress 2 1574
0.2515973150730133
[0.5133620500564575, 0.39571985602378845, 0.2515973150730133]
batch progress 3 1574
0.15495266020298004
[0.5133620500564575, 0.39571985602378845, 0.2515973150730133, 0.15495266020298004]
batch progress 4 1574
0.08430870622396469
[0.5133620500564575, 0.39571985602378845, 0.2515973150730133, 0.15495266020298004, 0.08430870622396469]
batch progress 5 1574
0.047250498086214066
[0.5133620500564575, 0.39571985602378845, 0.2515973150730133, 0.15495266020298004, 0.08430870622396469, 0.047250498086214066]
batch progress 6 1574
0.025643764063715935
[0.5133620500564575, 0.39571985602378845, 0.2515973150730133, 0.15495266020298004, 0.08430870622396469, 0.047250498086214066, 0.025643764063715935]
batch progress 7 1574
0.017067810520529747
[0.5133620500564575, 0.39571985602378845, 0.2515973150730133